In [39]:
from PIL import Image
import numpy as np
import tifffile
import re
import cv2
import matplotlib.pyplot as plt
def parse_value(text, keyword):
    pattern = re.escape(keyword) + r'\s*=\s*"([^"]*)"'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
def get_metadata(image_path):
    with tifffile.TiffFile(image_path) as tif:
        metadata = tif.pages[0].tags
        tif_tags = {}
        tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')

    s = tif_tags['XMP']
    # print(s)
    dwarp_data = "drone-dji:DewarpData"
    dwarp_data_para = parse_value(s, dwarp_data)
    dwarp_coefficients  = [float(x) for x in dwarp_data_para[11:].split(',')]
    calibrated_hmatrix = "drone-dji:DewarpHMatrix"
    calibrated_hmatrix = [float(x) for x in parse_value(s, calibrated_hmatrix).split(',')]
    print(len(calibrated_hmatrix), calibrated_hmatrix)
    calibrated_hmatrix = np.array(calibrated_hmatrix).reshape((3,3))
    print(calibrated_hmatrix)
    centre_x_para = 'drone-dji:CalibratedOpticalCenterX'
    centre_y_para = 'drone-dji:CalibratedOpticalCenterY'
    vignettingData_para = 'drone-dji:VignettingData'
    center_x = float(parse_value(s, centre_x_para))
    center_y = float(parse_value(s, centre_y_para))
    vignettingData_str = parse_value(s, vignettingData_para)
    vignetting_data = [float(x) for x in vignettingData_str.split(',')]  # Convert string to list of floats
    return center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix

def undistort_image(image_path, outpath_img, coefficients):
    center_x, center_y = coefficients[0], coefficients[1]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = coefficients[2]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = 2200.899902343750, 2200.219970703125, 10.609985351562, -6.575988769531, 0.008104680106, -0.042915198952, -0.000333522010, 0.000239991001, 0.000000000000
    matrix = np.array([[fx, 0, center_x+cx], [0, fy, center_y+cy], [0, 0, 1]])
    dist = np.array([k1, k2, p1, p2, k3])
    # print("md", matrix, dist)
    with Image.open(image_path) as img:
        w, h = img.size
        # img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        # print(img_norm.max(), img_norm.min())
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, dist, (w,h), 1, (w,h))
        dst = cv2.undistort(np.array(img), matrix, dist, newcameramtx)
        print(np.array_equal(np.array(img), dst))
        # print(dst.max(), dst.min())
        undistorted_image = Image.fromarray(dst)
        undistorted_image.save(outpath_img)

def apply_vignetting_correction(image_path,outpath_img):
    with Image.open(image_path) as img:
        width, height = img.size
        center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix = get_metadata(image_path)
        print(center_x, center_y)
        correction_img = np.zeros((height, width))
        for x in range(width):
            for y in range(height):
                r = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
                correction_value = sum([k * (r ** i) for i, k in enumerate((vignetting_data))]) + 1.0
                correction_img[y, x] = correction_value
        img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        corrected_img = img_norm * correction_img
        corrected_img = np.clip(corrected_img, 0, 255).astype(np.uint8)
        corrected_image = Image.fromarray(corrected_img)
        corrected_image.save(outpath_img)
    return (center_x, center_y, dwarp_coefficients, calibrated_hmatrix)

def phase_alignment(image_path,outpath_img, parameters):
    calibrated_hmatrix = parameters[-1]
    calibrated_hmatrix = np.array([[9.891065e-01, 1.740813e-02, -1.592078e+01],
                                   [-1.568817e-02, 9.885082e-01, 3.766531e+01],
                                   [1.083204e-06, 5.127963e-07, 1.000000e+00]])
    with Image.open(image_path) as img:
        w, h = img.size
        print(w, h)
        transformed_image = cv2.warpPerspective(np.array(img), calibrated_hmatrix, (w, h))
        print(np.array_equal(np.array(img), transformed_image))
        transformed_image = Image.fromarray(transformed_image)
        transformed_image.save(outpath_img)

def ecc_alignment(image1_path, image2_path):
    image1 = cv2.imread(image1_path, cv2.IMREAD_UNCHANGED)
    image2 = cv2.imread(image2_path, cv2.IMREAD_UNCHANGED)
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
    aligned_image2 = cv2.warpPerspective(image2, M, (image1.shape[1], image1.shape[0]))
    print(np.array_equal(aligned_image2, image2))
    print('final_{image2_path}')
    cv2.imwrite('final_' + image2_path, aligned_image2)



# image_path = r"images\DJI_20230928151039_0001_MS_R.TIF"
# outpath_img = r"DJI_20230928151039_0001_MS_R.TIF"
# parameters = apply_vignetting_correction(image_path,outpath_img)
# undistort_image(outpath_img, 'undistort_'+outpath_img, parameters)
# phase_alignment('undistort_'+outpath_img, 'phase_undistort_'+outpath_img, parameters)
ecc_alignment(r'NIR\phase_undistort_DJI_20230928151039_0001_MS_NIR.TIF', r'R\phase_undistort_DJI_20230928151039_0001_MS_R.TIF')

False
final_{image2_path}


In [ ]:
img = cv2.imread('left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png',dst)